This notebook is based on my previous notebook [Baseline_For_Beginners](https://www.kaggle.com/code/adamlogman/baseline-for-beginners),and [Dmitry Uarov](https://www.kaggle.com/dmitryuarov) 's [NeurIPS | Baseline + External data](https://www.kaggle.com/code/dmitryuarov/neurips-baseline-external-data) notebook with modifications to the model

# Import Dependencies 

In [1]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
!pip install mordred --no-index --find-links=file:///kaggle/input/mordred-1-2-0-py3-none-any/

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
Looking in links: file:///kaggle/input/mordred-1-2-0-py3-none-any/
Processing /kaggle/input/mordred-1-2-0-py3-none-any/mordred-1.2.0-py3-none-any.whl
Processing /kaggle/input/mordred-1-2-0-py3-none-any/networkx-2.8.8-py3-none-any.whl (from mordred)
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but yo

In [2]:
 # Importing Required Libraries\nLet's begin by importing the essential Python libraries needed for data processing, visualization, and modeling.

import pandas as pd
import numpy as np


from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from mordred import Calculator, descriptors

import networkx as nx
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit import Chem

from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [3]:
# class CFG:
#     TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
#     SEED = 42
#     FOLDS = 10

In [4]:
# useless_cols = [   
    
#     'MaxPartialCharge', 
#     # Nan data
#     'BCUT2D_MWHI',
#     'BCUT2D_MWLOW',
#     'BCUT2D_CHGHI',
#     'BCUT2D_CHGLO',
#     'BCUT2D_LOGPHI',
#     'BCUT2D_LOGPLOW',
#     'BCUT2D_MRHI',
#     'BCUT2D_MRLOW',

#     # Constant data
#     'NumRadicalElectrons',
#     'SMR_VSA8',
#     'SlogP_VSA9',
#     'fr_barbitur',
#     'fr_benzodiazepine',
#     'fr_dihydropyridine',
#     'fr_epoxide',
#     'fr_isothiocyan',
#     'fr_lactam',
#     'fr_nitroso',
#     'fr_prisulfonamd',
#     'fr_thiocyan',

#     # High correlated data >0.95
#     'MaxEStateIndex',
#     'HeavyAtomMolWt',
#     'ExactMolWt',
#     'NumValenceElectrons',
#     'Chi0',
#     'Chi0n',
#     'Chi0v',
#     'Chi1',
#     'Chi1n',
#     'Chi1v',
#     'Chi2n',
#     'Kappa1',
#     'LabuteASA',
#     'HeavyAtomCount',
#     'MolMR',
#     'Chi3n',
#     'BertzCT',
#     'Chi2v',
#     'Chi4n',
#     'HallKierAlpha',
#     'Chi3v',
#     'Chi4v',
#     'MinAbsPartialCharge',
#     'MinPartialCharge',
#     'MaxAbsPartialCharge',
#     'FpDensityMorgan2',
#     'FpDensityMorgan3',
#     'Phi',
#     'Kappa3',
#     'fr_nitrile',
#     'SlogP_VSA6',
#     'NumAromaticCarbocycles',
#     'NumAromaticRings',
#     'fr_benzene',
#     'VSA_EState6',
#     'NOCount',
#     'fr_C_O',
#     'fr_C_O_noCOO',
#     'NumHDonors',
#     'fr_amide',
#     'fr_Nhpyrrole',
#     'fr_phenol',
#     'fr_phenol_noOrthoHbond',
#     'fr_COO2',
#     'fr_halogen',
#     'fr_diazo',
#     'fr_nitro_arom',
#     'fr_phos_ester'
# ]

SEED = 42

# Read Files

### Main Files

In [5]:
 #We will load both the training and test datasets using pandas, and store test IDs 
#train=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
ss=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')
ID=test['id']

In [6]:
mols_test = [Chem.MolFromSmiles(s) for s in test.SMILES]

# Initialize the Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True) # ignore_3D=True for 2D descriptors

desc_test = calc.pandas(mols_test)
desc_test.head()

100%|██████████| 3/3 [00:00<00:00,  4.47it/s]


ABC  \
0  module 'numpy' has no attribute 'float'.\n`np....   
1  module 'numpy' has no attribute 'float'.\n`np....   
2  module 'numpy' has no attribute 'float'.\n`np....   

                                               ABCGG  nAcid  nBase    SpAbs_A  \
0  module 'numpy' has no attribute 'float'.\n`np....      0      0  51.369068   
1  module 'numpy' has no attribute 'float'.\n`np....      0      0  53.698919   
2  module 'numpy' has no attribute 'float'.\n`np....      0      0  60.526875   

    SpMax_A  SpDiam_A     SpAD_A   SpMAD_A   LogEE_A     VE1_A     VE2_A  \
0  2.608687  5.217373  51.369068  1.252904  4.633213  4.054325  0.098886   
1  2.429583  4.859165  53.698919  1.309730  4.643161  4.685471  0.114280   
2  2.564154  4.983326  60.526875  1.315802  4.764144  4.942681  0.107450   

      VE3_A          VR1_A         VR2_A      VR3_A  nAromAtom  nAromBond  \
0  2.810771  138187.404971   3370.424511  13.247353         24         24   
1  2.955453     981.187389     23.931400   8.299750         30         30   
2  3.123964  637481.849542  13858.301077  14.891337         24         24   

   nAtom  nHeavyAtom  nSpiro  nBridgehead  nHetero  nH  nB  nC  nN  nO  nS  \
0     59          41       0            0       12  18   0  29   2   2   0   
1     67          41       0            0        6  26   0  35   0   4   0   
2     76          46       0            0       10  30   0  36   2   6   0   

   nP  nF  nCl  nBr  nI  nX  ATS0dv  ATS1dv  ATS2dv  ATS3dv  ATS4dv  ATS5dv  \
0   0   6    0    0   0   6   754.0   647.0  1100.0  1133.0  1481.0   988.0   
1   0   0    0    0   0   0   534.0   554.0   760.0   789.0   700.0   672.0   
2   0   0    0    0   0   0   648.0   655.0   974.0  1023.0   953.0   805.0   

   ATS6dv  ATS7dv  ATS8dv  ATS0d  ATS1d  ATS2d  ATS3d  ATS4d  ATS5d  ATS6d  \
0   788.0   662.0   564.0  234.0  291.0  429.0  490.0  466.0  427.0  353.0   
1   648.0   576.0   550.0  244.0  303.0  463.0  534.0  512.0  485.0  465.0   
2   797.0   722.0   743.0  276.0  355.0  550.0  599.0  573.0  552.0  534.0   

   ATS7d  ATS8d                                            ATS0s  \
0  311.0  294.0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  414.0  361.0  missing intrinsic state for ['*'] (ATS0s/Props)   
2  515.0  470.0  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS1s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS2s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS3s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS4s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS5s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS6s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS7s  \
0  missing intrinsic state for ['*'] (ATS0s/Props)   
1  missing intrinsic state for ['*'] (ATS0s/Props)   
2  missing intrinsic state for ['*'] (ATS0s/Props)   

                                             ATS8s   ATS0Z   ATS1Z   ATS2Z  \
0  missing intrinsic state for ['*'] (ATS0s/Props)  1774.0  1789.0  2888

In [7]:
# tc_smiles =pd.read_csv('/kaggle/input/tc-smiles/Tc_SMILES.csv',      low_memory=False)
# tg_smiles =pd.read_csv('/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv',      low_memory=False)
# ktg_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_tg3.xlsx')
# de_smiles =pd.read_excl('/kaggle/input/smiles-extra-data/data_dnst1.xlsx')
# ds1_smiles=pd.read_csv('/kagegle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv',      low_memory=False)
# ds2_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv',      low_memory=False)
# ds3_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv',      low_memory=False)
# ds4_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv',      low_memory=False)

tg_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_tg.csv',      low_memory=False)
tc_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_tc.csv',      low_memory=False)
rg_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_rg.csv',      low_memory=False)
ffv_ds     = pd.read_csv('/kaggle/input/modred-dataset/desc_ffv.csv',     low_memory=False)
den_ds     = pd.read_csv('/kaggle/input/modred-dataset/desc_de.csv',      low_memory=False)

### Extra Files

# Preprocessing 

In [8]:
for i in (tg_ds,tc_ds,rg_ds,ffv_ds,den_ds):
     i.drop(columns=[col for col in i.columns if i[col].nunique() == 1],axis=1,inplace=True)

tg_ds = tg_ds.select_dtypes(exclude=['object', 'category'])
rg_ds = rg_ds.select_dtypes(exclude=['object', 'category'])
ffv_ds = ffv_ds.select_dtypes(exclude=['object', 'category'])
tc_ds = tc_ds.select_dtypes(exclude=['object', 'category'])
den_ds = den_ds.select_dtypes(exclude=['object', 'category'])

In [9]:
# tg_ds.head()

In [10]:
# def make_smile_canonical(smile): # To avoid duplicates, for example: canonical '*C=C(*)C' == '*C(=C*)C'
#     try:
#         mol = Chem.MolFromSmiles(smile)
#         canon_smile = Chem.MolToSmiles(mol, canonical=True)
#         return canon_smile
#     except:
#         return np.nan

# train['SMILES'] = train['SMILES'].apply(lambda s: make_smile_canonical(s))
# test['SMILES'] = test['SMILES'].apply(lambda s: make_smile_canonical(s))

In [11]:
# ktg_smiles.rename(columns={'Tg [K]': 'Tg'}, inplace=True)
# tg_smiles.rename(columns={'Tg (C)': 'Tg'}, inplace=True)
# tc_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)
# de_smiles.rename(columns={'density(g/cm3)': 'Density'}, inplace=True)
# ds1_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)

In [12]:
# de_smiles['SMILES'] = de_smiles['SMILES'].apply(lambda s: make_smile_canonical(s))
# de_smiles = de_smiles[(de_smiles['SMILES'].notnull())&(de_smiles['Density'].notnull())&(de_smiles['Density'] != 'nylon')]
# de_smiles['Density'] = de_smiles['Density'].astype('float64')
# de_smiles['Density'] -= 0.118

# ktg_smiles['Tg'] = ktg_smiles['Tg'] - 273.15

In [13]:
# def preprocessing(df):
#     desc_names = [desc[0] for desc in Descriptors.descList if desc[0] not in useless_cols]
#     descriptors = [compute_all_descriptors(smi) for smi in df['SMILES'].to_list()]

#     graph_feats = {'graph_diameter': [], 'avg_shortest_path': [], 'num_cycles': []}
#     for smile in df['SMILES']:
#          compute_graph_features(smile, graph_feats)
        
#     result = pd.concat(
#         [
#             pd.DataFrame(descriptors, columns=desc_names),
#             pd.DataFrame(graph_feats)
#         ],
#         axis=1
#     )

#     result = result.replace([-np.inf, np.inf], np.nan)
#     return result

# Feature Extraction 

In [14]:
# # SMILESを正規化（canonical化）
# # 重複する SMILES を集約して平均 target 値に
# # df_train に存在するが欠損している target に、df_extra から値を補完

# def add_extra_data(df_train, df_extra, target):
#     n_samples_before = len(df_train[df_train[target].notnull()])
    
#     df_extra['SMILES'] = df_extra['SMILES'].apply(lambda s: make_smile_canonical(s))
#     df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()
#     cross_smiles = set(df_extra['SMILES']) & set(df_train['SMILES'])
#     unique_smiles_extra = set(df_extra['SMILES']) - set(df_train['SMILES'])

#     # Make priority target value from competition's df
#     for smile in df_train[df_train[target].notnull()]['SMILES'].tolist():
#         if smile in cross_smiles:
#             cross_smiles.remove(smile)

#     # Imput missing values for competition's SMILES
#     for smile in cross_smiles:
#         df_train.loc[df_train['SMILES']==smile, target] = df_extra[df_extra['SMILES']==smile][target].values[0]
    
#     df_train = pd.concat([df_train, df_extra[df_extra['SMILES'].isin(unique_smiles_extra)]], axis=0).reset_index(drop=True)

#     n_samples_after = len(df_train[df_train[target].notnull()])
#     print(f'\nFor target "{target}" added {n_samples_after-n_samples_before} new samples!')
#     print(f'New unique SMILES: {len(unique_smiles_extra)}')
#     return df_train

# train = add_extra_data(train, tc_smiles, 'Tc')
# train = add_extra_data(train, tg_smiles, 'Tg')
# train = add_extra_data(train, ktg_smiles, 'Tg')
# train = add_extra_data(train, de_smiles, 'Density')

# train = add_extra_data(train, ds1_smiles, 'Tc')
# #train = add_extra_data(train, ds2_smiles, 'Density')
# train = add_extra_data(train, ds3_smiles, 'Tg')
# train = add_extra_data(train, ds4_smiles, 'FFV')

In [15]:
# def compute_all_descriptors(smiles):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol is None:
#         return [None] * len(desc_names)
#     return [desc[1](mol) for desc in Descriptors.descList if desc[0] not in useless_cols]

# def compute_graph_features(smiles, graph_feats):
#     mol = Chem.MolFromSmiles(smiles)
#     adj = rdmolops.GetAdjacencyMatrix(mol)
#     G = nx.from_numpy_array(adj)

#     graph_feats['graph_diameter'].append(nx.diameter(G) if nx.is_connected(G) else 0)
#     graph_feats['avg_shortest_path'].append(nx.average_shortest_path_length(G) if nx.is_connected(G) else 0)
#     graph_feats['num_cycles'].append(len(list(nx.cycle_basis(G))))

# train = pd.concat([train, preprocessing(train)], axis=1)
# test = pd.concat([test, preprocessing(test)], axis=1)

# # Find constant columns for each target
# all_features = train.columns[7:].tolist()
# features = {}
# for target in CFG.TARGETS:
#     const_descs = []
#     for col in train.columns.drop(CFG.TARGETS):
#         if train[train[target].notnull()][col].nunique() == 1:
#             const_descs.append(col)
#     features[target] = [f for f in all_features if f not in const_descs]

# print(train.shape)
# train['Ipc']=np.log10(train['Ipc'])  
# for n in train.columns[7:]:
#     train[n]=train[n].replace(-np.inf,np.nan)
#     train[n]=train[n].replace(np.inf,np.nan)    
#     train[n].fillna(train[n].mean())
  
# print(train.shape)
# test['Ipc']=np.log10(test['Ipc'])
# for n in test.columns[7:]:
#     train[n]=train[n].replace(-np.inf,np.nan)
#     train[n]=train[n].replace(np.inf,np.nan)      
#     test[n].fillna(train[n].mean())

In [16]:
# train

# Data Preparation For Model Training 

In [17]:
# # We'll separate train to be one model for each target variable.
# t_1=train[['SMILES','Tg']].copy()
# t_2=train[['SMILES','FFV']].copy()
# t_3=train[['SMILES','Tc']].copy()
# t_4=train[['SMILES','Density']].copy()
# t_5=train[['SMILES','Rg']].copy()

# # We will drop the rows with missing values related to that target after separation.
# #This is important , dropping them beforehand would result Null for all data.
# t_1.dropna(inplace=True)
# t_2.dropna(inplace=True)
# t_3.dropna(inplace=True)
# t_4.dropna(inplace=True)
# t_5.dropna(inplace=True)

In [18]:
# train=train.drop(['id','Tg','FFV','Tc','Density','Rg'],axis=1)
test=test.drop(['id','SMILES'],axis=1)

In [19]:
# tg=t_1.merge(train,on='SMILES',how='left')
# ffv=t_2.merge(train,on='SMILES',how='left')
# tc=t_3.merge(train,on='SMILES',how='left')
# density=t_4.merge(train,on='SMILES',how='left')
# rg=t_5.merge(train,on='SMILES',how='left')

In [20]:
# for i in (tg,tc,density,ffv,rg):
#     i.drop('SMILES',axis=1,inplace=True)
#     i.dropna(inplace=True)

# Model 

In [21]:

def model(train_d,test_d,model,target,submission=False):
    # We divide the data into training and validation sets for model evaluation
    train_cols = set(train_d.columns) - {target}
    test_cols = set(test_d.columns)
   # Intersect the feature columns
    common_cols = list(train_cols & test_cols)
    X=train_d[common_cols].copy()
    y=train_d[target].copy()
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

    Model=model()
    if submission==False:
       Model.fit(X_train,y_train)
       y_pred=Model.predict(X_test)
       return mean_absolute_error(y_pred,y_test)         # We assess our model performance using MAE metric
    if submission==True:
       Model.fit(X,y)
       submission=Model.predict(test_d[common_cols].copy())
       return submission

# '''
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error
# import numpy as np

# def model(train_d, test_d, model, target, submission=False, debug=False):
#     """
#     train_d, test_d : pandas DataFrame
#     model            : either a callable that returns an estimator (e.g. lambda: LGBMRegressor(...))
#                        OR an already-instantiated estimator (e.g. LGBMRegressor(...))
#                        OR a class (e.g. LGBMRegressor) — callable(class) will instantiate with default args.
#     target           : name of target column in train_d
#     submission       : if True, fit on full train and return predictions for test_d
#     debug            : if True, prints extra debug info
#     """
#     # find shared numeric columns (features)
#     train_cols = set(train_d.columns) - {target}
#     test_cols = set(test_d.columns)
#     common_cols = sorted(list(train_cols & test_cols))

#     if debug:
#         print(f"[model] target={target} common feature count={len(common_cols)}")
#         print("  first few common cols:", common_cols[:10])
#         print("  train shape:", train_d.shape, "test shape:", test_d.shape)

#     if len(common_cols) == 0:
#         if debug:
#             print("  No common feature columns between train and test -> skipping.")
#         return np.nan if not submission else np.array([])

#     # build X, y (drop rows with NaN target)
#     X = train_d[common_cols].copy()
#     y = train_d[target].copy()
#     valid_idx = ~y.isna()
#     X = X.loc[valid_idx].reset_index(drop=True)
#     y = y.loc[valid_idx].reset_index(drop=True)

#     if X.shape[0] == 0:
#         if debug:
#             print("  No training rows with non-null target.")
#         return np.nan if not submission else np.array([])

#     # split for quick CV evaluation
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)

#     # Accept either callable (factory/class) or an estimator instance
#     if hasattr(model, "fit") and callable(getattr(model, "fit")):
#         # it's already an estimator instance (has fit method)
#         est = model
#     else:
#         # try to instantiate (model might be a callable factory or class)
#         est = model()

#     if submission is False:
#         # train on split and return MAE
#         est.fit(X_train, y_train)
#         y_pred = est.predict(X_val)
#         return mean_absolute_error(y_val, y_pred)
#     else:
#         # train on full train and predict test (preserve same column order)
#         est.fit(X, y)
#         X_test = test_d[common_cols].copy()
#         # try to convert object columns to numeric if any slipped in
#         for c in X_test.select_dtypes(include=['object','category']).columns:
#             try:
#                 X_test[c] = X_test[c].astype(float)
#             except Exception:
#                 X_test = X_test.drop(columns=[c])
#                 if debug:
#                     print("  dropped non-numeric test col:", c)
#         preds = est.predict(X_test)
#         return preds
# '''

# **Model Evaluation

In [22]:

models_to_compare = {
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(
        max_iter=1000,
        learning_rate=0.05,
        max_leaf_nodes=31,
        early_stopping='auto',
        random_state=SEED
    ),
    'XGBoost': lambda: XGBRegressor(
        # tuned for FFV
        n_estimators=800,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.7,
        reg_alpha=0.0,
        reg_lambda=1,
        tree_method='auto',   # change to 'gpu_hist' if GPU-compatible wheel and GPU available
        random_state=SEED,
        verbosity=0,
        n_jobs=-1
    ),
    'LightGBM': lambda: LGBMRegressor(
        # tuned for Tc
        n_estimators=500,
        learning_rate=0.01,
        num_leaves=64,
        subsample=0.6,
        colsample_bytree=0.5,
        reg_alpha=0.001,
        reg_lambda=1,
        random_state=SEED,
        n_jobs=-1,
        verbose=-1
    ),
    'CatBoost': lambda: CatBoostRegressor(
        # tuned for Tg
        iterations=1000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        early_stopping_rounds=50,
        random_seed=SEED,
        verbose=0,
        allow_writing_files=False
    ),
    'HistGradientBoosting2': lambda: HistGradientBoostingRegressor(),
    'XGBoost2': lambda: XGBRegressor(verbosity=0),
    'LightGBM2': lambda: LGBMRegressor(verbose=-1),
    'CatBoost2': lambda: CatBoostRegressor(verbose=0),
}

In [23]:
# scaler = StandardScaler()
# tg["Tg"] = scaler.fit_transform(np.array(tg["Tg"]).reshape(-1, 1))

# for name, model_init in models_to_compare.items():
#     try:
#         score = model(tg_ds, desc_test, model_init, 'Tg', submission=False)
#         print(f'{name:<21}: MAE = {score:.5f}')
#     except Exception as e:
#         print(f'{name:<21}: Failed with error: {e}')

In [24]:

# for name, model_cls in models_to_compare.items():
#     try:
#         score = model(ffv_ds, desc_test, model_cls, 'FFV', submission=False)
#         print(f'{name:<21}: MAE = {score:.5f}')
#     except Exception as e:
#         print(f'{name:<21}: Failed with error: {e}')

In [25]:

# for name, model_cls in models_to_compare.items():
#     try:
#         score = model(tc_ds, desc_test, model_cls, 'Tc', submission=False)
#         print(f'{name:<21}: MAE = {score:.5f}')
#     except Exception as e:
#         print(f'{name:<21}: Failed with error: {e}')

In [26]:

# for name, model_cls in models_to_compare.items():
#     try:
#         score = model(den_ds, desc_test, model_cls, 'Density', submission=False)
#         print(f'{name:<21}: MAE = {score:.5f}')
#     except Exception as e:
#         print(f'{name:<21}: Failed with error: {e}')

In [27]:

# for name, model_cls in models_to_compare.items():
#     try:
#         score = model(rg_ds, desc_test, model_cls, 'Rg', submission=False)
#         print(f'{name:<21}: MAE = {score:.5f}')
#     except Exception as e:
#         print(f'{name:<21}: Failed with error: {e}')

# Final Model For Submission

In [28]:

'''
11297.2s	273	=== Best models summary ===
11297.2s	274	Tg: CatBoost  CV_MAE=42.369263  params={'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'depth': 6}
11297.2s	275	FFV: XGBoost  CV_MAE=0.006229  params={'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 0.0, 'n_estimators': 800, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.7}
11297.2s	276	Tc: LightGBM  CV_MAE=0.032098  params={'subsample': 0.6, 'reg_lambda': 1, 'reg_alpha': 0.001, 'num_leaves': 64, 'n_estimators': 500, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
11297.2s	277	Density: GradientBoosting  CV_MAE=0.065598  params={'subsample': 0.8, 'n_estimators': 800, 'max_features': 'sqrt', 'max_depth': 3, 'learning_rate': 0.1}
11297.2s	278	Rg: GradientBoosting  CV_MAE=1.717341  params={'subsample': 0.8, 'n_estimators': 1000, 'max_features': 'sqrt', 'max_depth': 5, 'learning_rate': 0.01}
'''

def predict(train, test, target):
    return (model(train, test, CatBoostRegressor, target, submission=True)*0.6+
            model(train, test, XGBRegressor, target, submission=True)*0.4)/1
sub = {
    'id' : ID,
    'Tg': predict(tg_ds, desc_test, 'Tg'),
    'FFV': predict(ffv_ds, desc_test, 'FFV'),
    'Tc' : predict(tc_ds, desc_test, 'Tc'),
    'Density' : predict(den_ds, desc_test, 'Density'),
    'Rg' : predict(rg_ds, desc_test, 'Rg')
}

# sub={'id':ID,'Tg':model(tg_ds,desc_test,CatBoostRegressor,'Tg',submission=True),
#      'FFV':model(ffv_ds,desc_test,CatBoostRegressor,'FFV',submission=True),
#      'Tc':model(tc_ds,desc_test,CatBoostRegressor,'Tc',submission=True),
#      'Density':model(den_ds,desc_test,CatBoostRegressor,'Density',submission=True),
#      'Rg':model(rg_ds,desc_test,CatBoostRegressor,'Rg',submission=True)}

Learning rate set to 0.058055
0:	learn: 114.7379024	total: 145ms	remaining: 2m 24s
1:	learn: 110.4948992	total: 202ms	remaining: 1m 40s
2:	learn: 106.4904193	total: 258ms	remaining: 1m 25s
3:	learn: 102.8094916	total: 314ms	remaining: 1m 18s
4:	learn: 99.3141175	total: 370ms	remaining: 1m 13s
5:	learn: 96.0052376	total: 427ms	remaining: 1m 10s
6:	learn: 92.7895706	total: 486ms	remaining: 1m 8s
7:	learn: 89.9857768	total: 543ms	remaining: 1m 7s
8:	learn: 87.4459092	total: 601ms	remaining: 1m 6s
9:	learn: 84.9479419	total: 658ms	remaining: 1m 5s
10:	learn: 82.6943582	total: 714ms	remaining: 1m 4s
11:	learn: 80.4861910	total: 770ms	remaining: 1m 3s
12:	learn: 78.2817303	total: 825ms	remaining: 1m 2s
13:	learn: 76.3275825	total: 883ms	remaining: 1m 2s
14:	learn: 74.6461039	total: 938ms	remaining: 1m 1s
15:	learn: 73.0636352	total: 993ms	remaining: 1m 1s
16:	learn: 71.5547472	total: 1.05s	remaining: 1m
17:	learn: 70.0257588	total: 1.11s	remaining: 1m
18:	learn: 68.6301616	total: 1.16s	remai

In [29]:
submission=pd.DataFrame(sub)

In [30]:
submission

,id,Tg,FFV,Tc,Density,Rg
0,1109053969,159.975532,0.371701,0.186090,1.164030,20.437211
1,1422188626,162.111278,0.377923,0.233897,1.088670,20.394124
2,2032016830,154.175872,0.353964,0.233747,1.110439,20.554671


In [31]:
submission.to_csv('submission.csv',index=False)